## Import Libs

In [6]:
import pandas as pd
import ast
from collections import Counter
import json
from langdetect import detect

## Carregando Dataset

In [7]:
df = pd.read_json("../gliner.json")
pd.reset_option('display.max_colwidth')

In [8]:
df.head()

,tokenized_text,ner
0,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,..."
1,"[A, Cônsul, Britânico, em, Brasília, ,, Sir, W...","[[6, 8, person], [17, 19, person], [36, 41, CP..."
2,"[Il, pompiere, Saif, Al, Khaili, ,, di, nazion...","[[2, 4, person], [68, 70, date], [76, 82, addr..."
3,"[Aquela, queria, ser, imam, em, uma, mesquita,...","[[25, 26, person], [24, 26, name], [31, 31, pa..."
4,"[Le, dossier, médical, de, Mme, Kabila, Nzanzu...","[[4, 6, person], [10, 12, birth date], [73, 80..."


Percebemos que ele foi treinado com diversos idiomas, então no Dataset tem ingles, portugues, espanhol, etc

Ambas colunas são listas e podemos "explodir" para ter entender melhor

In [9]:
df.shape

(19635, 2)

In [10]:
df.columns

Index(['tokenized_text', 'ner'], dtype='object')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19635 entries, 0 to 19634
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tokenized_text  19635 non-null  object
 1   ner             19635 non-null  object
dtypes: object(2)
memory usage: 306.9+ KB


## Data Preparation

### Deixando cada valor da lista como uma nova linha

In [12]:
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

df['tokenized_text'] = df['tokenized_text'].apply(safe_literal_eval)
df['ner'] = df['ner'].apply(safe_literal_eval)

In [13]:
def map_tokens_with_entities(row):
    tokens = row['tokenized_text']
    entities = row['ner']

    mapped_tokens = []

    for idx, token in enumerate(tokens):
        token_entity = None
        for start, end, entity_type in entities:
            if start <= idx < end:
                token_entity = entity_type
                break

        mapped_tokens.append((token, token_entity))

    return mapped_tokens

df['mapped_tokens'] = df.apply(map_tokens_with_entities, axis=1)

In [14]:
exploded_df = df.explode('mapped_tokens', ignore_index=True)
exploded_df[['token', 'entity_type']] = pd.DataFrame(exploded_df['mapped_tokens'].tolist(), index=exploded_df.index)
exploded_df = exploded_df.drop(columns=['mapped_tokens'])

In [15]:
exploded_df

,tokenized_text,ner,token,entity_type
0,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,...",Mamadou,person
1,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,...",Diop,None
2,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,...",",",None
3,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,...",a,None
4,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,...",resident,None
...,...,...,...,...
2100247,"[A, hospedagem, ', Riad, Alhambra, ', ,, local...","[[3, 4, organization], [26, 27, person], [42, ...",cobrir,None
2100248,"[A, hospedagem, ', Riad, Alhambra, ', ,, local...","[[3, 4, organization], [26, 27, person], [42, ...",eventuais,None
2100249,"[A, hospedagem, ', Riad, Alhambra, ', ,, local...","[[3, 4, organization], [26, 27, person], [42, ...",despesas,None
2100250,"[A, hospedagem, ', Riad, Alhambra, ', ,, local...","[[3, 4, organization], [26, 27, person], [42, ...",médicas,None


In [16]:
exploded_df.shape[0]

2100252

In [17]:
exploded_df['entity_type'].isnull().sum()

np.int64(1751395)

Palavras que não foram identificadas como PII viraram uma linha None, filtrar essas linhas

In [18]:
df = exploded_df.dropna(subset=['entity_type'])

In [19]:
df.shape[0]

348857

In [20]:
df['entity_type'].value_counts()

entity_type
person                    48872
address                   48216
email                     37452
phone number              33302
organization              25253
                          ...  
dose of insulin               1
nationality ID number         1
passport series number        1
account name                  1
health issue                  1
Name: count, Length: 2896, dtype: int64

Existem entidades que não fazem parte da lista de labels de PII -> Filtrando o df com apenas as labels de PII

In [21]:
all_labels = ["person","organization","phone number","address","passport number","email","credit card number","social security number","health insurance id number","date of birth","mobile phone number","bank account number","medication","cpf","driver's license number","tax identification number","medical condition","identity card number","national id number","ip address","email address","iban","credit card expiration date","username","health insurance number","registration number","student id number","insurance number","flight number","landline phone number","blood type","cvv","reservation number","digital signature","social media handle","license plate number","cnpj","postal code","passport_number","serial number","vehicle registration number","credit card brand","fax number","visa number","insurance company","identity document number","transaction number","national health insurance number","cvc","birth certificate number","train ticket number","passport expiration date","social_security_number"]

In [22]:
df = df[df['entity_type'].isin(all_labels)]

In [23]:
df.shape[0]

241486

In [24]:
df.head()

,tokenized_text,ner,token,entity_type
0,"[Mamadou, Diop, ,, a, resident, of, Dakar, ,, ...","[[0, 1, person], [14, 15, nightclub], [45, 45,...",Mamadou,person
66,"[A, Cônsul, Britânico, em, Brasília, ,, Sir, W...","[[6, 8, person], [17, 19, person], [36, 41, CP...",Sir,person
67,"[A, Cônsul, Britânico, em, Brasília, ,, Sir, W...","[[6, 8, person], [17, 19, person], [36, 41, CP...",William,person
77,"[A, Cônsul, Britânico, em, Brasília, ,, Sir, W...","[[6, 8, person], [17, 19, person], [36, 41, CP...",Sir,person
78,"[A, Cônsul, Britânico, em, Brasília, ,, Sir, W...","[[6, 8, person], [17, 19, person], [36, 41, CP...",John,person


## Análise

In [25]:
df['entity_type'].value_counts()

entity_type
person                              48872
address                             48216
email                               37452
phone number                        33302
organization                        25253
credit card number                  13097
date of birth                       10169
mobile phone number                  7587
social security number               3649
email address                        2358
medication                           1840
passport number                      1582
bank account number                  1049
medical condition                    1009
credit card expiration date           764
ip address                            681
tax identification number             598
health insurance number               577
health insurance id number            482
identity card number                  464
driver's license number               460
landline phone number                 408
fax number                            244
passport expiration da

Existe uma discrepância de treinamento entre alguns PII's, por exemplo 'cpf' foi treinado com 8 exemplos, enquanto 'person' com 48872

In [26]:
df[df['entity_type'] == 'medication']['token'].value_counts()

token
Metformin     97
mg            79
,             76
/             64
500           53
              ..
omeprazolo     1
pillola        1
Cimetidin      1
Telfast        1
IU             1
Name: count, Length: 452, dtype: int64

OBS: Parece que como ele separa em tokens, quando uma palavra é composta, ele separa ela e treina cada palavra. Por exemplo: 'Metformin 500.9mg', ele separa e cada parte é treinada, então 'Metformin' é considerado medicamento e está correto, porém '500', ',' e 'mg' também é treinado como medicamento.

In [27]:
df[df['entity_type'] == 'social_security_number']['token'].value_counts()

token
.            5
Security     3
de           3
2211         3
Social       3
sécurité     2
numéro       2
456          2
123          2
SW           1
342516789    1
INS          1
202          1
881          1
12           1
345          1
sociale      1
número       1
seguro       1
Name: count, dtype: int64

In [28]:
df[df['entity_type'] == 'social security number']['token'].value_counts()

token
.       1141
123      336
456      285
de       108
111       63
        ... 
1112       1
5512       1
Li         1
578        1
KZ         1
Name: count, Length: 412, dtype: int64

In [29]:
df[df['entity_type'] == 'email']['token'].value_counts()

token
.                 12307
@                  6341
gmail              2151
hotmail             295
schmidt             233
                  ...  
mangala               1
porto                 1
kigali                1
musoni                1
kpomassiekodjo        1
Name: count, Length: 6178, dtype: int64

In [30]:
df[df['entity_type'] == 'email address']['token'].value_counts()

token
.                  767
@                  403
gmail              234
hotmail             39
email               27
                  ... 
family               1
libyagovernment      1
jordi                1
abdulla              1
marti                1
Name: count, Length: 567, dtype: int64

## Visualizar os textos de treino com as entidades encontradas

### Preparando

In [31]:
def safe_eval(row):
    # Check if the row is already a list
    if isinstance(row, list):
        return row # If it's a list, return it directly
    try:
        # If it's a string, try to decode it as JSON
        return json.loads(row.replace("'", "\""))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for row: {row}")
        print(e)
        return []

all_entities = [ent[2] for row in df['ner'] for ent in safe_eval(row)]
entity_counts = Counter(all_entities)
print(entity_counts)

Counter({'person': 455801, 'organization': 167503, 'phone number': 152462, 'address': 150913, 'email': 120848, 'passport number': 101135, 'credit card number': 71308, 'social security number': 70630, 'location': 61149, 'date': 51223, 'date of birth': 50815, 'birth date': 49692, 'health insurance ID number': 40826, 'city': 28195, 'mobile phone number': 27319, 'bank account number': 23428, 'medication': 22953, 'CPF': 19101, 'company registration number': 16951, 'birthdate': 13716, "driver's license number": 13604, 'tax identification number': 12848, 'health insurance number': 11139, 'health insurance id number': 10922, 'medical condition': 10000, 'currency': 7930, 'identity card number': 7451, 'age': 7436, 'personal identification number': 6603, 'email address': 6560, 'country': 6107, 'tax ID number': 5860, 'national ID number': 5172, 'date_of_birth': 5151, 'credit card expiration date': 5057, 'IBAN': 4970, 'product': 4537, 'tax ID': 4418, 'bank': 4333, 'condition': 4119, 'credit card se

In [32]:
def reconstruct_text(row):
    # Check if the 'tokenized_text' is already a list
    # If it's a string, then use ast.literal_eval
    tokens = row['tokenized_text'] if isinstance(row['tokenized_text'], list) else ast.literal_eval(row['tokenized_text'])

    # Check if the 'ner' is already a list
    # If it's a string, then use ast.literal_eval
    entities = row['ner'] if isinstance(row['ner'], list) else ast.literal_eval(row['ner'])

    annotated_text = tokens.copy()

    for start, end, entity_type in entities:
        annotated_text[start] = f"{tokens[start]}({entity_type})"

    return ' '.join(annotated_text)

df['annotated_text'] = df.apply(reconstruct_text, axis=1)

In [33]:
pd.set_option('display.max_colwidth', None)

### Visualizando

In [34]:
df['annotated_text'].drop_duplicates(keep='first')

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        Mamadou(person) Diop , a resident of Dakar , is a regular at the ' Club(nightclub) 54 ' nightclub located at 45 Rue de la Liberté . He usually arrives around midnight and often leaves around 3 am . Mamadou ' s ID card number is WS-123456789-1(ID card number) , and he pays for his drinks with a credit card : 1234-5678-9012-3456(credit card number) .
66                                                                                                                                                              

### Removendo as linhas que não são com os idiomas pt, en, es

In [35]:
df['texto'] = df['tokenized_text'].apply(lambda x: ' '.join(x))

In [36]:
df['texto']

0                                                                                                                                                                                                                                                                                                                                      Mamadou Diop , a resident of Dakar , is a regular at the ' Club 54 ' nightclub located at 45 Rue de la Liberté . He usually arrives around midnight and often leaves around 3 am . Mamadou ' s ID card number is WS-123456789-1 , and he pays for his drinks with a credit card : 1234-5678-9012-3456 .
66                                                                                                                                          A Cônsul Britânico em Brasília , Sir William Percival , recebeu recentemente uma solicitação de passaporte de Sir John Watson , residente em Campinas , São Paulo . O pedido incluiu documentos de identificação , como CPF 123 . 456 . 789-0

In [37]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [ ]:
df['idioma'] = df['texto'].apply(detect_language)

In [ ]:
df['idioma'].unique()

In [ ]:
df.to_csv("/content/drive/MyDrive/gliner-dataset-prep.csv", index=False)

In [ ]:
df['idioma'].value_counts()

,count
idioma,
en,45735
de,41840
fr,40882
pt,40027
es,38438
it,34557
ca,6
tr,1


In [ ]:
filtered_df = df[df['language'].isin(['en', 'es', 'pt'])]

### Labels que não fazem sentido no nosso Caso de Uso

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/gliner-dataset-prep.csv")

In [ ]:
df.head()

,Unnamed: 0,tokenized_text,ner,token,entity_type,annotated_text,texto,idioma
0,0,"['Mamadou', 'Diop', ',', 'a', 'resident', 'of', 'Dakar', ',', 'is', 'a', 'regular', 'at', 'the', ""'"", 'Club', '54', ""'"", 'nightclub', 'located', 'at', '45', 'Rue', 'de', 'la', 'Liberté', '.', 'He', 'usually', 'arrives', 'around', 'midnight', 'and', 'often', 'leaves', 'around', '3', 'am', '.', 'Mamadou', ""'"", 's', 'ID', 'card', 'number', 'is', 'WS-123456789-1', ',', 'and', 'he', 'pays', 'for', 'his', 'drinks', 'with', 'a', 'credit', 'card', ':', '1234-5678-9012-3456', '.']","[[0, 1, 'person'], [14, 15, 'nightclub'], [45, 45, 'ID card number'], [58, 58, 'credit card number']]",Mamadou,person,"Mamadou(person) Diop , a resident of Dakar , is a regular at the ' Club(nightclub) 54 ' nightclub located at 45 Rue de la Liberté . He usually arrives around midnight and often leaves around 3 am . Mamadou ' s ID card number is WS-123456789-1(ID card number) , and he pays for his drinks with a credit card : 1234-5678-9012-3456(credit card number) .","Mamadou Diop , a resident of Dakar , is a regular at the ' Club 54 ' nightclub located at 45 Rue de la Liberté . He usually arrives around midnight and often leaves around 3 am . Mamadou ' s ID card number is WS-123456789-1 , and he pays for his drinks with a credit card : 1234-5678-9012-3456 .",en
1,66,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília', ',', 'Sir', 'William', 'Percival', ',', 'recebeu', 'recentemente', 'uma', 'solicitação', 'de', 'passaporte', 'de', 'Sir', 'John', 'Watson', ',', 'residente', 'em', 'Campinas', ',', 'São', 'Paulo', '.', 'O', 'pedido', 'incluiu', 'documentos', 'de', 'identificação', ',', 'como', 'CPF', '123', '.', '456', '.', '789-01', ',', 'CNH', '0123', '.', '456', '.', '789-12', 'e', 'RG', '11', '.', '223', '.', '554-0', '.', 'Além', 'disso', ',', 'apresentou', 'duas', 'fotos', 'recentes', ',', 'um', 'cartão', 'de', 'crédito', 'para', 'verificação', 'de', 'identidade', 'e', 'endereço', 'de', 'residência', 'atual', ':', 'Rua', 'José', 'de', 'Anchieta', ',', '123', ',', 'Jardim', 'Oceânico', ',', 'Campinas', ',', 'SP', '.']","[[6, 8, 'person'], [17, 19, 'person'], [36, 41, 'CPF'], [43, 48, 'CNH'], [50, 55, 'RG'], [79, 91, 'address'], [79, 91, 'current address']]",Sir,person,"A Cônsul Britânico em Brasília , Sir(person) William Percival , recebeu recentemente uma solicitação de passaporte de Sir(person) John Watson , residente em Campinas , São Paulo . O pedido incluiu documentos de identificação , como CPF(CPF) 123 . 456 . 789-01 , CNH(CNH) 0123 . 456 . 789-12 e RG(RG) 11 . 223 . 554-0 . Além disso , apresentou duas fotos recentes , um cartão de crédito para verificação de identidade e endereço de residência atual : Rua(current address) José de Anchieta , 123 , Jardim Oceânico , Campinas , SP .","A Cônsul Britânico em Brasília , Sir William Percival , recebeu recentemente uma solicitação de passaporte de Sir John Watson , residente em Campinas , São Paulo . O pedido incluiu documentos de identificação , como CPF 123 . 456 . 789-01 , CNH 0123 . 456 . 789-12 e RG 11 . 223 . 554-0 . Além disso , apresentou duas fotos recentes , um cartão de crédito para verificação de identidade e endereço de residência atual : Rua José de Anchieta , 123 , Jardim Oceânico , Campinas , SP .",pt
2,67,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília', ',', 'Sir', 'William', 'Percival', ',', 'recebeu', 'recentemente', 'uma', 'solicitação', 'de', 'passaporte', 'de', 'Sir', 'John', 'Watson', ',', 'residente', 'em', 'Campinas', ',', 'São', 'Paulo', '.', 'O', 'pedido', 'incluiu', 'documentos', 'de', 'identificação', ',', 'como', 'CPF', '123', '.', '456', '.', '789-01', ',', 'CNH', '0123', '.', '456', '.', '789-12', 'e', 'RG', '11', '.', '223', '.', '554-0', '.', 'Além', 'disso', ',', 'apresentou', 'duas', 'fotos', 'recentes', ',', 'um', 'cartão', 'de', 'crédito', 'para', 'verificação', 'de', 'identidade', 'e', 'endereço', 'de', 'residência', 'atual', ':', 'Rua', 'José', 'de', 'Anchieta', ',', '123', ',', 'Jard

In [ ]:
df = df[df['idioma'].isin(['en', 'es', 'pt'])]

In [ ]:
df
pd.reset_option('display.max_colwidth')

In [ ]:
df.head()

,Unnamed: 0,tokenized_text,ner,token,entity_type,annotated_text,texto,idioma
0,0,"['Mamadou', 'Diop', ',', 'a', 'resident', 'of'...","[[0, 1, 'person'], [14, 15, 'nightclub'], [45,...",Mamadou,person,"Mamadou(person) Diop , a resident of Dakar , i...","Mamadou Diop , a resident of Dakar , is a regu...",en
1,66,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",Sir,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt
2,67,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",William,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt
3,77,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",Sir,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt
4,78,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",John,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt


In [ ]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
df.head()

,tokenized_text,ner,token,entity_type,annotated_text,texto,idioma
0,"['Mamadou', 'Diop', ',', 'a', 'resident', 'of'...","[[0, 1, 'person'], [14, 15, 'nightclub'], [45,...",Mamadou,person,"Mamadou(person) Diop , a resident of Dakar , i...","Mamadou Diop , a resident of Dakar , is a regu...",en
1,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",Sir,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt
2,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",William,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt
3,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",Sir,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt
4,"['A', 'Cônsul', 'Britânico', 'em', 'Brasília',...","[[6, 8, 'person'], [17, 19, 'person'], [36, 41...",John,person,"A Cônsul Britânico em Brasília , Sir(person) W...","A Cônsul Britânico em Brasília , Sir William P...",pt


In [ ]:
df['entity_type'].unique()

array(['person', 'address', 'phone number', 'email', 'organization',
       'tax identification number', "driver's license number",
       'mobile phone number', 'email address', 'credit card number',
       'medication', 'social security number',
       'credit card expiration date', 'passport number', 'date of birth',
       'reservation number', 'medical condition', 'social media handle',
       'health insurance id number', 'bank account number', 'ip address',
       'health insurance number', 'postal code', 'blood type', 'username',
       'insurance company', 'landline phone number', 'flight number',
       'insurance number', 'license plate number',
       'passport expiration date', 'identity document number',
       'identity card number', 'registration number', 'fax number',
       'vehicle registration number', 'passport_number',
       'social_security_number', 'digital signature', 'cpf',
       'credit card brand', 'national id number',
       'birth certificate number', '

In [ ]:
len(df['entity_type'].value_counts())

46

In [ ]:
len(all_labels)

53

In [ ]:
diferenca = [item for item in all_labels if item not in df['entity_type'].unique()]

Valores na lista que não estão na coluna: ['student id number', 'cvv', 'cnpj', 'transaction number', 'national health insurance number', 'cvc', 'train ticket number']


In [ ]:
for i in diferenca:
  print(i)

student id number
cvv
cnpj
transaction number
national health insurance number
cvc
train ticket number


In [ ]:
for i in all_labels:
  print(i)

person
organization
phone number
address
passport number
email
credit card number
social security number
health insurance id number
date of birth
mobile phone number
bank account number
medication
cpf
driver's license number
tax identification number
medical condition
identity card number
national id number
ip address
email address
iban
credit card expiration date
username
health insurance number
registration number
student id number
insurance number
flight number
landline phone number
blood type
cvv
reservation number
digital signature
social media handle
license plate number
cnpj
postal code
passport_number
serial number
vehicle registration number
credit card brand
fax number
visa number
insurance company
identity document number
transaction number
national health insurance number
cvc
birth certificate number
train ticket number
passport expiration date
social_security_number


### Labels que talvez não façam sentido no nosso Caso de Uso
- social security number (só quem nasce nos EUA que tem)
- fax number
- insurance company
- serial number
- credit card brand
- train ticket number
- flight number
- reservation number

### Labels que talvez estão "duplicadas"
- health insurance id number -> está mais ligado à identificação do beneficiário.<br>
health insurance number -> pode se referir ao contrato ou plano específico.

- email<br> email address

- identity card number -> Número do RG, DNI (Argentina)<br> national id number -> CPF (Brasil), SSN (EUA), NIN (Reino Unido)<br> identity document number -> Número de passaporte, Número do RG, Número de documento de identidade
- cvc<br> cvv
- passport number<br> passport_number